In [9]:
import pandas as pd
import csv
import numpy as np

filename = "data-raw/Tom LD _ Life Story Presentation.txt"

    # Specify model path
model_path = "GloVE/glove.6B/glove.6B.300d.txt"

words = pd.read_table(
    model_path, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE
)

model_df = pd.read_table(model_path, sep=" ", index_col=0, header=None, quoting=3)
model_dict = {
    word: embeddings for word, embeddings in zip(model_df.index, model_df.values)
}




In [40]:
from functions.embeddings_using_pd import get_word_embeddings

In [41]:
em = get_word_embeddings(filename, model_path)
em.shape

(14, 300)

In [24]:
def get_content(filename):
       # Open the input file and read its contents
    with open(filename, "r") as f:
        contents = f.read()

    # Split the contents into blocks based on integer line followed by a blank line
    blocks = contents.split("\n\n")

    return blocks

blocks = get_content(filename)


In [25]:
blocks[66]

'We are all just chemical reactions on a small rock flying through incomprehensible space\n'

In [27]:
# Iterate over the blocks and extract the word embeddings for each one
embeddings_list = []
for block in blocks:
    # Tokenize the block into individual words
    words = block.split()
    # Create an empty array to store the embeddings for each word
    embeddings = np.zeros((len(words), len(model_dict[next(iter(model_dict))])))
    # Iterate over the words and get their embeddings
    for i, word in enumerate(words):
        if word in model_dict:
            embeddings[i] = model_dict[word]

    # Calculate the mean embedding for the block
    mean_embedding = np.mean(embeddings, axis=0)
    embeddings_list.append(mean_embedding)



In [39]:
np.array(embeddings_list).shape

(67, 300)

In [ ]:
# def main():
#     # Specify file path
#     filename = "data-raw/Tom LD _ Life Story Presentation.txt"

#     # Specify model path
#     model_path = "GloVE/glove.6B/glove.6B.300d.txt"

#     embeddings_list = get_word_embeddings(filename, model_path)

#     print(embeddings_list.head())

In [49]:
import numpy as np
from sklearn.manifold import TSNE
import graphviz as gv
from PIL import Image


def reduce_and_draw_network_map_with_images(
    embeddings, image_folder, handle_isolates=True
):
    # Perform t-SNE to reduce the embeddings to 2 dimensions
    tsne = TSNE(n_components=2, random_state=42, perplexity=5)
    embeddings_2d = tsne.fit_transform(embeddings)

    # Find isolated nodes if requested
    if handle_isolates:
        degrees = np.sum(np.abs(np.sign(np.dot(embeddings, embeddings.T))), axis=1)
        isolated_nodes = np.where(degrees == 0)[0]
    else:
        isolated_nodes = []

    # Create a Graphviz graph object
    graph = gv.Graph(engine="neato")

    # Add nodes to the graph
    for i in range(len(embeddings_2d)):
        # Load image
        img_path = f"{image_folder}/Slide{i+1}.jpeg"
        # img = Image.open(img_path)  # .resize((50, 50))

        # Add node with image
        with graph.subgraph(name=f"cluster_{i}") as c:
            c.attr(label=f"{i}")
            c.attr(fontsize="10")
            c.attr(style="filled")
            c.attr(color="black")
            c.node(f"node{i}", image=img_path, shape="none", label="")

            # Set position of cluster
            if i in isolated_nodes:
                c.attr(pos=f"{2*embeddings_2d[i,0]},{2*embeddings_2d[i,1]}!")
            else:
                c.attr(pos=f"{embeddings_2d[i,0]},{embeddings_2d[i,1]}!")

    # Add edges to the graph
    for i in range(len(embeddings_2d)):
        for j in range(i + 1, len(embeddings_2d)):
            similarity = np.dot(embeddings[i], embeddings[j]) / (
                np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[j])
            )
            if similarity > 0.8:
                graph.edge(f"node{i}", f"node{j}", weight=str(similarity))

    # Draw the network map
    graph.format = "pdf"
    graph.render(filename="network_map_with_images 6")


In [50]:
reduce_and_draw_network_map_with_images(np.array(embeddings_list[1:]), 'img', handle_isolates=False)

/Users/zellaking/.pyenv/versions/3.9.9/envs/keras/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/zellaking/.pyenv/versions/3.9.9/envs/keras/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
